## Serpent

docs:

- https://github.com/ethereum/wiki/wiki/Serpent
- https://mc2-umd.github.io/ethereumlab/docs/serpent_tutorial.pdf

Serpent is one of the high-level programming languages used to write Ethereum contracts. The language, as suggested by its name, is designed to be very similar to Python; it is intended to be maximally clean and simple, combining many of the efficiency benefits of a low-level language with ease-of-use in programming style, and at the same time adding special domain-specific features for contract programming.

Serpent have some limitations compared to Python:

- Number wrapping: In Serpent numbers wrap aroung 2^256, for example the expression 3^(2^254)) evaluates to 1, while in Python numbers have potentially unlimited size.
- No decimals
- No list comprehensions
- No first-class fucntions. There is no concept of persistence between function calls.
- No persistent storage
- `extern` statement, used to call functions from other contracts

### Installation

The easiest way to install Serpent in thourh the Pip management package system:

```bash
pip install ehtereum-serpent
```

In order to test Serpent programs, install also the `ethereum` package:

```bash
pip install ethereum
```

Pyethereum tester: https://github.com/ethereum/pyethereum/wiki/Using-pyethereum.tester

In [237]:
import serpent
import ethereum.tools.tester as t
import ethereum.abi as abi
import serpent
import binascii
import re
import sys

is_string = lambda x: isinstance(x, (str, str, bytes))

## Test

In [238]:
# program = open("./serpent_contracts/thousands.se").read()
# machine_code = serpent.compile(program)
# # blockchain = t.State(0)
# c = t.Chain()
# contract = c.contract(program, language='serpent')
# res = contract.get_confirmation()
# res = serpent.enc(res)
# if is_string(res):
#     res = res.rstrip('\x00')

Initializing chain from provided state


In [259]:
# def test_evm():
#     evm_code = serpent.compile(serpent_code)
#     translator = abi.ContractTranslator(serpent.mk_full_signature(
#                                         serpent_code))
#     data = translator.encode('main', [2, 5])
#     s = tester.state()
#     c = s.evm(evm_code)
#     o = translator.decode('main', s.send(tester.k0, c, 0, data))
#     assert o == [32]

## Manage Timestamps 

Here there is solidity reference

You can access the current block time using `block.timestamp` ('now' is a synonym of this). The time will be returned as a POSIX timestamp (basically the monotonous number of seconds since 1970-01-01 00:00:00 UTC).

However, this timestamp is 'set' by the miner that ends up mining your transaction. As such, the miner can manipulate the timestamp. There are certain rules to this; other parties will not accept a block if the given timestamp occurs in the future, for instance (more detail about this). But this does mean that the number cannot be used for e.g. random number generation.

If you want someone to only perform an action after one hour, you can at an earlier action (such as when the contract is created), store the current time, e.g.: `action_time = now;`, and then in another call, perform a check like: `require (now > (action_time + 1 hour));`

## Testing Basics

Introduce the `pyethereum.tester` package.

https://github.com/ethereum/pyethereum/wiki/Using-pyethereum.tester

Explain the component, that it provides keys and addresses etc...

In [246]:
CONTRACT = "./serpent_contracts/tests.se"
program = open(CONTRACT).read()
machine_code = serpent.compile(program)

The `c = t.Chain()` line initializes a new state (ie. a genesis block). The `c.contract(...)` line creates a new contract, and creates an object in Python which represents it. You can use x.address to access this contract's address. The fourth line calls the contract with argument 42, and we see 84 predictably come out.

In [247]:
c = t.Chain()
contract = c.contract(program, language='serpent')
contract_address = contract.address

def parse_function(s):
    # s = "get_confirmation()"
    method       = s[:s.find("(")]
    args         = s[s.find("(") + 1:-1]
    args         = [int(e, 32) for e in args.split(",") if e]
    call_data    = serpent.mk_full_signature(program)
    call_data    = [e for e in call_data if e["name"].startswith(method + "(")]
    return method, args, call_data

Initializing chain from provided state


With `serpent.mk_full_signature` we can see the entire ABI **???** signature of the contract, i.e. the interface we can use to interact with the contract.
For each function is defined its `name`, `inputs` and `outputs` types.

In [248]:
serpent.mk_full_signature(program)

[{'name': 'print_int()',
  'type': 'function',
  'constant': False,
  'inputs': [],
  'outputs': [{'name': 'out', 'type': 'int256'}]},
 {'name': 'print_string()',
  'type': 'function',
  'constant': False,
  'inputs': [],
  'outputs': [{'name': 'out', 'type': 'int256'}]},
 {'name': 'print_string_arg(int256)',
  'type': 'function',
  'constant': False,
  'inputs': [{'name': 's', 'type': 'int256'}],
  'outputs': [{'name': 'out', 'type': 'int256'}]},
 {'name': 'return_sum(int256,int256)',
  'type': 'function',
  'constant': False,
  'inputs': [{'name': 'input1', 'type': 'int256'},
   {'name': 'input2', 'type': 'int256'}],
  'outputs': [{'name': 'out', 'type': 'int256'}]},
 {'name': 'test_contract()',
  'type': 'function',
  'constant': False,
  'inputs': [],
  'outputs': [{'name': 'out', 'type': 'int256'}]}]

In [249]:
# translator object useful to quickly translate value usign the contract's ABI specification
translator = abi.ContractTranslator(serpent.mk_full_signature(program))

Function `ContractTranslator.encode()`: returns the encoded function name and arguments so that it can be used with the evm to execute a funcion call, the binary string follows the Ethereum Contract ABI.

We can now test some functions by calling the ABI interface of the contrat.

Remember:
   
- `integer` argument: no need of encoding
- `real` (`float`) argument
- `string` argument: encode to 32bit integer using `int(arg, 32)`

#### Test  `print_int()`

In [250]:
call_data = translator.encode('print_int', [])
res = c.tx(sender=t.k0, to=contract_address, value=0, data=call_data)
print("Hex result:\t{}".format(res))

Hex result:	b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\xe8'


As you can see, the results is in hex format. The contract will treat all values as .... explain how the values are treated. How strings are converted etc...

We can solve this calling `translator.decode` to decode the result according to the contract's ABI return value.

In [251]:
res = translator.decode('print_int', res)
print("Decoded result:\t{}".format(res))

Decoded result:	[1000]


#### Test `print_string()`

In [256]:
call_data = translator.encode('print_string', [])
res = c.tx(sender=t.k0, to=contract_address, value=0, data=call_data)
print("Hex result:\t{}".format(res))

Hex result:	b'It Works!\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'


In [257]:
res = translator.decode('print_string', res)
print("Decoded result:\t{}".format(res))

Decoded result:	[33224015419227930193771281274863023050005528119172647639991667569939917570048]


Ops! It seems that the decoding function converts our string to an integer representation. TODO: Go to see impl of decode()

Notice the return value of the `print_string` function of the contract's ABI. 

``` 
{
 'name': 'print_string()', 
 'type': 'function',
 'constant': False,
 'inputs': [], 
 'outputs': [{'name': 'out', 'type': 'int256'}]
}
```

The `outputs` field is of type `int256`. As you remember from **???**  Serpent treat short string as integers...

#### Test `print_string_arg(arg1)`

In [194]:
call_data = translator.encode('print_string_arg', ["Hello World!"])
res = c.tx(sender=t.k0, to=contract_address, value=0, data=call_data)
print("Hex result:\t{}".format(res))

Hex result:	b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00Hello World!'


In [195]:
res = translator.decode('print_int', res)
print("Decoded result:\t{}".format(res))

Decoded result:	[22405534230753928650781647905]


#### Test `return_sum(arg1, arg2)`

In [198]:
call_data = translator.encode('return_sum', [5, 7])
res = c.tx(sender=t.k0, to=contract_address, value=0, data=call_data)
print("Hex result:\t{}".format(res))

Hex result:	b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0c'


In [199]:
res = translator.decode('print_int', res)
print("Decoded result:\t{}".format(res))

Decoded result:	[12]


## Test long_string.se

In [21]:
MY_STRING = "Saving a string longer than 32 bytes!"

**What to do in case we want to retun a string LONGER than 32 bytes? Try with `save()` and `load()`**

In [22]:
program = open("./serpent_contracts/long_string.se").read()
machine_code = serpent.compile(program)
# blockchain = t.State(0)
c = t.Chain()
contract = c.contract(program, language='serpent')
res = contract.saveTitle(0, MY_STRING)
res = contract.getTitle(0)
print(res)

Initializing chain from provided state
b'This is a string longer than 32 bytes, of 51 bytes!'


## Crowdfunding

In [205]:
recipient = t.a0
goal = 10  # 10 ether
timelimit = 10 * 24 * 3600  # 10 days in seconds

In [204]:
recipient

b'\x82\xa9x\xb3\xf5\x96*[\tW\xd9\xee\x9e\xefG.\xe5[B\xf1'

In [234]:
CONTRACT = "./crowdfunding/my_cf.se"
program = open(CONTRACT).read()
machine_code = serpent.compile(program)
c = t.Chain()
contract = c.contract(program, language='serpent')
contract_address = contract.address
translator = abi.ContractTranslator(serpent.mk_full_signature(program))

def send(chain, translator, sender, to, method, args, value=0):
    call_data = translator.encode(method, args)
    res = chain.tx(sender=sender, to=to, value=value, data=call_data)
    print("Hex result:\t{}".format(res))
    if res != b'':
        res = translator.decode('contribute', res)
    else:
        res = ''
    print("Decoded result:\t{}".format(res))

Initializing chain from provided state


In [235]:
send(c, translator, t.k0, contract_address, "create_campaign", [recipient, goal, timelimit])

Hex result:	b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x82\xa9x\xb3\xf5\x96*[\tW\xd9\xee\x9e\xefG.\xe5[B\xf1'
Decoded result:	[745948140856946866108753121277737810491401257713]


In [236]:
send(c, translator, t.k1, contract_address, "contribute", [], value=9)

Hex result:	b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\t'
Decoded result:	[9]
